Initializing:

In [0]:
import numpy
import random
#!pip install pycuda
import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule

Matrix initialization and util functions:

Matrices are square.

In [0]:
global m1
global m2
global height1
global height2
global width1
global width2
global res 

height1 = 4
width1 = 4
height2 = 4
width2 = 4

m1 = numpy.zeros((height1,width1)).astype(numpy.int32)
m2 = numpy.zeros((height2,width2)).astype(numpy.int32)
res = numpy.zeros((height1,width2)).astype(numpy.int32)

for i in range(height1):
  for j in range(width1):
    m1[i][j] = random.randrange(0, 10)

for i in range(height2):
  for j in range(width2):
    m2[i][j] = random.randrange(0, 10)

def print_matrix(matrix):
  for i in range(len(matrix)):
    for j in range(len(matrix[i])):
        print(matrix[i][j],end = "  ")
    print("\n")

def matrix_mul(matrix1,matrix2):
  return matrix1.dot(matrix2)

# CUDA kernel
Matrix multiplication using CUDA shared memory. All threads of a block copy the needed matrix chunks into shared memory for their block and then proceed to work from there.

In [0]:
mod = SourceModule("""
     __global__ void multiplyMatrix(int *m1, int *m2, int *res, int width){
       
       #define TILE 2

       __shared__ int m1shared[TILE][TILE];
       __shared__ int m2shared[TILE][TILE];

       int col = blockIdx.x * TILE + threadIdx.x;
       int row = blockIdx.y * TILE + threadIdx.y;
       int idx = row*width + col;

       int range = (width / TILE) + ((width % TILE) != 0);

       
          int val = 0;
          
          for(int i = 0; i<range; ++i){
            if (row < width && i*TILE+threadIdx.x < width)
             m1shared[threadIdx.y][threadIdx.x] = m1[row*width+(i*TILE + threadIdx.x)];
            else
             m1shared[threadIdx.y][threadIdx.x] = 0;
            if (col < width && i*TILE+threadIdx.y < width)
             m2shared[threadIdx.y][threadIdx.x] = m2[(i*TILE + threadIdx.y)*width + col];
            else
             m2shared[threadIdx.y][threadIdx.x] = 0;
            
            __syncthreads();
          
            for(int j=0; j<TILE; ++j){
                val += m1shared[threadIdx.y][j]*m2shared[j][threadIdx.x];
            }
          
            __syncthreads();
          
          }

          if( (row < width) && (col < width))
        {
          res[idx] = val;
        }
     }
   """)

Main:

In [16]:
print_matrix(m1)
print_matrix(m2)

m1_gpu = cuda.mem_alloc(m1.nbytes)
m2_gpu = cuda.mem_alloc(m2.nbytes)
res_gpu = cuda.mem_alloc(res.nbytes)

cuda.memcpy_htod(m1_gpu, m1)
cuda.memcpy_htod(m2_gpu, m2)
cuda.memcpy_htod(res_gpu, res)

# tiles - we divide the matrix into tiles
t_width = 2

# kernel
func = mod.get_function("multiplyMatrix")
func(m1_gpu, m2_gpu, res_gpu, numpy.int32(width1), block=(t_width,t_width,1), grid=(4, 4, 1))

cuda.memcpy_dtoh(res, res_gpu)

print_matrix(res)

7  5  8  2  

1  2  0  4  

0  3  8  8  

3  3  5  1  

4  0  7  6  

2  0  9  2  

6  6  2  6  

5  1  5  6  

96  50  120  112  

28  4  45  34  

94  56  83  102  

53  31  63  60  

96  50  120  112  

28  4  45  34  

94  56  83  102  

53  31  63  60  

